### imports

In [ ]:
import torch
from gnn_temporal import RecurrentGCN
from torch_geometric_temporal import ChickenpoxDatasetLoader
from torch_geometric_temporal import temporal_signal_split


### Get data

In [ ]:
loader = ChickenpoxDatasetLoader()
dataset = loader.get_dataset()

train, test = temporal_signal_split(dataset, train_ratio=0.9)

### tarining

In [ ]:
model = RecurrentGCN(node_features=8, filters=32)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

model.train()

for epoch in range(200):
    cost = 0
    for time, snapshot in enumerate(train):
        y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr)
        cost = cost + torch.mean((y_hat-snapshot.y)**2)
    cost = cost / (time+1)
    cost.backward()
    optimizer.step()
    optimizer.zero_grad()

### eval

In [ ]:
model.eval()
cost = 0
for time, snapshot in enumerate(test):
    y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr)
    cost = cost + torch.mean((y_hat-snapshot.y)**2)
cost = cost / (time+1)
cost = cost.item()
print("MSE: {:.4f}".format(cost))